In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys

In [ ]:
sys.path.append('/content/drive/MyDrive')
import statistics
import numpy as np
import importlib
import fncs
import matplotlib.pyplot as plt
import random
importlib.reload(fncs)
from scipy import stats as st
import pandas as pd
from keras.utils import to_categorical
from collections import Counter
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten, Reshape
from keras.utils import to_categorical
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.optimizers import SGD, Adam, RMSprop
from keras.layers import Dropout, InputLayer, LSTM
from keras.layers import Bidirectional, BatchNormalization
from keras.layers import TimeDistributed
from keras.layers import Conv1D, AveragePooling1D, Dropout, UpSampling1D, BatchNormalization, LeakyReLU
from keras import regularizers

In [ ]:
def makeXandY(xt, xv, yt, yv):
  count = 0
  xLabels = []
  for i in range(len(yt)):
    while((count <= len(xt)) and (xt[count] <= yt[i])):
      xLabels.append(yv[i])
      count+=1


  dfxv = pd.DataFrame(xv)
  dfxLabel = pd.DataFrame(xLabels)

  xandy = pd.concat([dfxv, dfxLabel], axis = 1)

  xandy = xandy.dropna()


  return xandy





In [ ]:
def loadFeatures(dataFolder,idList):
    df = None
    for k,id in enumerate(idList):
        # Loading the raw data
        xt, xv, yt, yv = fncs.loadTrial(dataFolder,id=id)
        tempDf = makeXandY(xt, xv, yt, yv)
        if df is None:
          df = tempDf
        else:
          df = pd.concat([df, tempDf], ignore_index=True)


    return df.values[:, :6], df.values[:, 6]

In [ ]:

dirTrain = '/content/drive/MyDrive/train/train/'

valIDs = [11,1,15]
trainIDs = list(set(np.array(range(29))+1).difference(valIDs))

xTrain, yTrain = loadFeatures(dirTrain, trainIDs)
xVal, yVal = loadFeatures(dirTrain, valIDs)

In [ ]:
def encoding(y):
  return to_categorical(y, num_classes=4).reshape(-1,4)

def normalize(x):
  scaler = StandardScaler()
  return scaler.fit_transform(x)


In [ ]:
xTrain = normalize(xTrain)
xVal = normalize(xVal)


In [ ]:
yTrain = encoding(yTrain)
yVal = encoding(yVal)

In [ ]:
def windowDefinition(windowSize, x, y):
  windowX = [x[i:i+windowSize] for i in range(0, len(x)-windowSize, windowSize+1)]
  windowY = [y[i:i+windowSize] for i in range(0, len(y)-windowSize, windowSize+1)]

  windowX = np.array(windowX).reshape(-1, windowSize, 6)
  windowY = np.array(windowY).reshape(-1, windowSize, 4)

  return windowX, windowY


In [ ]:
xTrain, yTrain = windowDefinition(40, xTrain, yTrain)
xVal, yVal = windowDefinition(40, xVal, yVal)

xTrain.shape


In [ ]:
yTrain.shape

In [ ]:
print(xTrain.shape, yTrain.shape, xVal.shape, yVal.shape)

In [ ]:
yTrain

In [ ]:
def create_1dCNN():
  model = Sequential()
  model.add(Conv1D(16, kernel_size=3, activation='relu', padding='same', input_shape=(40,6)))

  model.add(MaxPooling1D(2))
  model.add(Dropout(0.2))

  model.add(Conv1D(32, kernel_size=3, activation='relu', padding='same'))
  model.add(MaxPooling1D(2))
  model.add(Dropout(0.2))

  model.add(Conv1D(32, kernel_size=3, activation='relu', padding='same'))
  model.add(UpSampling1D(size=2))
  model.add(Dropout(0.2))

  model.add(Conv1D(16, kernel_size=3, activation='relu', padding='same'))
  model.add(UpSampling1D(size=2))
  model.add(Dropout(0.2))

  model.add(Conv1D(4, kernel_size=3, activation='softmax', padding='same'))
  model.compile(optimizer=Adam(learning_rate=0.001),  loss='categorical_crossentropy', metrics = ['accuracy'])
  return model

In [ ]:
model = create_1dCNN()
model.summary()

In [ ]:
cnn_fit = model.fit(xTrain, yTrain, epochs=20, verbose=1, validation_data=(xVal, yVal))

In [ ]:
dirTest = '/content/drive/MyDrive/train/test/'

In [ ]:
def testWindows(testX, windowSize):
    windowX = [testX[i:i+windowSize] for i in range(0, len(testX)-windowSize+1, windowSize)]

    if len(testX) % windowSize != 0:
        windowX.append(testX[-windowSize:])

    windowX = np.array(windowX).reshape(-1, windowSize, 6)
    return windowX


In [ ]:
import random

def makeFinalRes(testxt, testxv, testyt, testyv):
  count = 0
  predictionsFinal = [0 for i in range(len(testyt))]
  for i in range(len(testyt)):
    vals = []
    while(count<len(testxt) and (testxt[count] <= testyt[i])):
      vals.append(testyv[count])
      count+=1

    if len(vals) == 0:
      predictionsFinal[i] = predictionsFinal[i-1]

    else:
      predictionsFinal[i] = statistics.mode(vals)




  return np.array(predictionsFinal)


In [ ]:
def saveArraysAsCSV(array1, array2, filename):
    structuredArray = np.column_stack((array1, array2))

    np.savetxt(filename, structuredArray, delimiter=',')

In [ ]:
ids = [1,2,3,4]
finalPath = '/content/drive/MyDrive/ans/'
for id in ids:
  testxt, testxv, testyt, testyv = fncs.loadTrial(dirTest, id)
  testxv = normalize(testxv)
  testxv = testWindows(testxv, 40)
  testyv = model.predict(testxv)
  testyv = testyv.reshape(-1,4)
  testyv = testyv[:len(testxt)]
  testyv = np.argmax(testyv, axis = 1)
  finalPred = makeFinalRes(testxt, testxv, testyt, testyv)
  nameOfCSV = "Trial0" + str(id) + "_y.csv"
  saveArraysAsCSV(testyt, finalPred, finalPath + nameOfCSV)



